# Tratamiento de datos

In [1]:
import numpy as np
import pandas
import re
from data import DATA
from hsic import HSIC
import time
import matplotlib.pyplot as plt

## 1. Definición de constantes

In [2]:
TOT_FREQ_DATA_ORI = "../static/ctc/TWITTER_COLOMBIA_FREQ.csv"
TOT_FREQ_DATA_DES = "../static/ctc/TWITTER_COLOMBIA_FREQ_CLEAN.csv"
REL_FREQ_DATA_DES = "../static/ctc/TWITTER_COLOMBIA_RELATIVE_FREQ_CLEAN.csv"
RANK_DATA_DES = "../static/ctc/TWITTER_COLOMBIA_RANK.csv"
NOR_RANK_DES = "../static/ctc/TWITTER_COLOMBIA_RANK_NORM.csv"

# Datos geográficos
data = pandas.DataFrame(list(DATA), columns= ['Longitud', 'Latitud', 'radio', 'ciudad', 'dep'])
data = data.drop(['radio','dep'], axis=1)
data = data.sort_values(by='ciudad')
data = data.set_index('ciudad')

## 2. Funciones de tratamiento de archivos

In [3]:
def ValidWordsFrame(file):
    general_words_data = pandas.read_csv(
        file, 
        encoding='utf-8', 
        usecols=[*range(0,6)],
        skiprows=[*range(1,6)],
        dtype={'total':np.int32,'lower':np.int32, 'titled':np.int32, 'upper':np.int32, 'other':np.int32},
        index_col = ['#CITY#']
    )
    #filtrar las palabras con una sola aparición, porque dan hsic malos
    general_words_data=general_words_data.loc[general_words_data['total']>1]
    
    #filtrar acronimos
    general_words_data=general_words_data.loc[general_words_data['lower']>general_words_data['upper']]
    
    #filtrar posibles nombres propios
    general_words_data=general_words_data.loc[general_words_data['lower']>general_words_data['titled']]
    
    #obtener palabras válidas hasta aquí
    valid_words = pandas.DataFrame(general_words_data.index)
    
    
    # Filtrar otras cosas
    
    # Filtro de posibles links
    http_pat = re.compile(r'.*http.*',flags=re.IGNORECASE)
    valid_words['http'] = valid_words['#CITY#'].str.contains(http_pat)
    valid_words=valid_words.loc[valid_words['http']==False]

    # Filtro risas posibles links
    risa_pat = re.compile(r'.*(ja){1,}.*',flags=re.IGNORECASE)
    valid_words['risa'] = valid_words['#CITY#'].str.contains(risa_pat)
    valid_words=valid_words.loc[valid_words['risa']==False]
    
    # Filtro pics al final, porque no se que es pero me late que es algo de twiter, una foto posiblemente
    pic_pat = re.compile(r'.*pic$',flags=re.IGNORECASE)
    valid_words['pic'] = valid_words['#CITY#'].str.contains(pic_pat)
    valid_words=valid_words.loc[valid_words['pic']==False]
    
    # Filtro palabras con mucha repetición de letras (más de tres letras iguales repetidas)
    multiple_path = re.compile(r'(\w)\1{2,}',flags=re.IGNORECASE)
    valid_words['multiple'] = valid_words['#CITY#'].str.contains(multiple_path)
    valid_words=valid_words.loc[valid_words['multiple']==False]
    
    # Filtro palabras con numeros
    numeros_path = re.compile(r'.*(\d).*',flags=re.IGNORECASE)
    valid_words['numeros'] = valid_words['#CITY#'].str.contains(numeros_path)
    valid_words=valid_words.loc[valid_words['numeros']==False]
    valid_words=valid_words.set_index('#CITY#')
    
    
    return(valid_words.index)

In [4]:
def ValidCitiesFrame(file, minwords=50000):
    # Datos de las ciudades filtradas
    general_cities_data =pandas.read_csv(
        file, 
        encoding='utf-8',
        nrows=5,
        index_col = ['#CITY#']
    )
    general_cities_data.drop(labels=['total','lower', 'titled', 'upper','other'], axis=1, inplace=True)
    general_cities_data.drop(labels=['#COUNTRY#','#TWEETS#', '#USERS#'], axis=0, inplace=True)
    general_cities_data=general_cities_data.transpose()
    general_cities_data['#TOTAL_WORDS#']=general_cities_data['#TOTAL_WORDS#'].apply(pandas.to_numeric)
    # Agrego datos geográficos
    localidades_filtradas = pandas.concat([data, general_cities_data], axis=1)
    #Elimino los corpus pequeños
    localidades_filtradas = localidades_filtradas.loc[localidades_filtradas['#TOTAL_WORDS#']>=minwords]
    return (localidades_filtradas)

In [5]:
GeoValidCities=ValidCitiesFrame(TOT_FREQ_DATA_ORI).drop(['#TOTAL_WORDS#','#VOCABULARY_SIZE#'], axis=1).sort_index()
GeoValidCities=GeoValidCities.as_matrix()
GeoValidCities

array([[  1.805187  , -75.89711628],
       [  8.3060079 , -73.64677   ],
       [ 10.0347577 , -73.2546581 ],
       [ 11.1606614 , -72.5962945 ],
       [  5.6564209 , -75.8884088 ],
       [  7.866667  , -76.666667  ],
       [  7.084512  , -70.755366  ],
       [  4.533889  , -75.681111  ],
       [ 10.7919858 , -74.932762  ],
       [  6.4389568 , -75.4043117 ],
       [  5.9340873 , -73.6212408 ],
       [  7.065278  , -73.854722  ],
       [ 10.9838942 , -74.853037  ],
       [  5.2009242 , -75.8736495 ],
       [  4.649178  , -74.062827  ],
       [  4.3386473 , -76.1955393 ],
       [  9.973157  , -73.8974372 ],
       [  7.125393  , -73.119804  ],
       [  3.880099  , -77.031164  ],
       [  3.8991844 , -76.3346675 ],
       [  3.437222  , -76.5225    ],
       [ 10.3799647 , -74.8995316 ],
       [  7.7553751 , -76.6642077 ],
       [ 10.399722  , -75.514444  ],
       [  7.9786912 , -75.2153806 ],
       [  8.884229  , -75.8093401 ],
       [  3.7187411 , -75.4991709 ],
 

In [6]:
def FiltredWordLocationFrame(file,destino):
    ValidWords=ValidWordsFrame(file)
    ValidCities=ValidCitiesFrame(file)
    lenvalidcities=len(ValidCities)
    
    file_word_freq = pandas.read_csv(
        file, 
        encoding='utf-8',
        chunksize=10000, 
        index_col=0,
        skiprows=[1,3,5],
        sep =",", 
        quotechar='"'
    )
    
    i=0
    for chunk in file_word_freq:
        cities_to_drop = list(set(list(chunk.columns))-set(ValidCities.index))
        words_to_drop = list(set(list(chunk.index))-set(ValidWords))
        chunk.drop(labels=cities_to_drop, axis=1, inplace=True)
        chunk.drop(labels=words_to_drop, inplace=True)
        if i==0:
            filtredwordsfreq=chunk
        else:
            filtredwordsfreq=filtredwordsfreq.append(chunk)
        print('Loop {}. Borrados = {}, tamaño actual del arreglo: {}'.format(i, len(words_to_drop), len(filtredwordsfreq)))
        i+=1
    totales=pandas.DataFrame(filtredwordsfreq.sum(),columns=['#TOTAL_WORDS#']).transpose()
    filtredwordsfreq = pandas.concat([totales, filtredwordsfreq])    
            
    filtredwordsfreq.to_csv(destino, sep="\t",decimal=",",header=filtredwordsfreq.columns)
        
    print('Listo!!')   
    
    

In [7]:
def relative_freq_file(file,destino):
    file_word_freq = pandas.read_csv(
        file, 
        encoding='utf-8', 
        index_col=0,
        sep ="\t",
        decimal=",",
        quotechar='"'
    )
    frecuencias_relativas = file_word_freq.iloc[0:]/file_word_freq.iloc[0]
    frecuencias_relativas.to_csv(destino, sep="\t",decimal=",",header=frecuencias_relativas.columns)


In [8]:
def rank_file(file,destino):
    file_word_rank = pandas.read_csv(
        file, 
        encoding='utf-8', 
        index_col=0,
        skiprows=[1],
        sep ="\t",
        decimal=",",
        quotechar='"'
    )
    file_word_rank = file_word_rank.rank(method='dense',ascending=False)
    max_rank = file_word_rank.max(axis=0)
    file_word_rank = pandas.concat([pandas.DataFrame(max_rank, columns=['#MAX_RANK#']).transpose(), file_word_rank])    
    file_word_rank = file_word_rank.astype(int)
    file_word_rank.to_csv(destino, sep="\t",decimal=",",header=file_word_rank.columns)

In [9]:
def normaliced_rank_file(file,destino):
    normaliced_word_rank = pandas.read_csv(
        file, 
        encoding='utf-8', 
        index_col=0,
        sep ="\t",
        decimal=",",
        quotechar='"'
    )
    normaliced_word_rank = (normaliced_word_rank.iloc[0:]/normaliced_word_rank.iloc[0])*100000000
    normaliced_word_rank.to_csv(destino, sep="\t",decimal=",",header=normaliced_word_rank.columns)

## Cálculos de HSIC

In [10]:
origen = NOR_RANK_DES
carpeta = '/src/static/ctc/'
destino = "CTC_HSIG_SOBRE_RANK_NORMALIZADO"
extencion = ".csv"

def escribir_cabezote_archivo(destino, fieldnames):
    with open(destino, 'w') as csvfile:
        writer = fieldnames.to_csv(csvfile,sep="\t",decimal=",",header=fieldnames.columns)
        
def escribir_datos_archivo(destino, values):
    with open(destino, 'a') as csvfile:
        writer = values.to_csv(csvfile,sep="\t",decimal=",",header=False)     


In [11]:
def conditional_hsic_to_file(file, destino, desde, hasta):
    destino= '/src/static/ctc/{}_{}a{}.csv'.format(destino,desde,hasta)
    reader = pandas.read_csv(
        file, 
        encoding='utf-8',
        chunksize=1,
        index_col=0,
        skiprows=[1,*range(0,desde)],
        sep ="\t",
        decimal=",",
        quotechar='"'
    )
    valid_count = 0
    count=desde

    for chunk  in reader:
        if count <= hasta:    
            start_time = time.time()
            cho=np.log(chunk.transpose().sort_index().as_matrix())
            hsic, pvalue=HSIC.HSIC_conditional_pval(GeoValidCities,cho, 0.0015, N_samp=2000, kernelX="Gaussian", kernelY="Gaussian")
            if(hsic and pvalue<0.05):
                valid_count+=1
                if valid_count == 1:
                    basic_header = pandas.DataFrame(columns=['HSIC','PValue'])
                    basic_header.loc[chunk.index.values[0],'HSIC']=hsic
                    basic_header.loc[chunk.index.values[0],'PValue']=pvalue
                    escribir_cabezote_archivo(destino, basic_header)
                    elapsed_time = int(time.time() - start_time)
                    print ('\rDuracion pros palabra:{}\t procesados:{},validados:{}'.format(elapsed_time,count,valid_count),end='\t\t\t\t\t\t\t\t\t\r')
                else:
                    basic_header = pandas.DataFrame(columns=['HSIC','PValue'])
                    basic_header.loc[chunk.index.values[0],'HSIC']=hsic
                    basic_header.loc[chunk.index.values[0],'PValue']=pvalue
                    escribir_datos_archivo(destino, basic_header)
                    elapsed_time = int(time.time() - start_time)
                    print ('\rDuracion pros palabra:{}\t procesados:{},validados:{}'.format(elapsed_time,count,valid_count),end='\t\t\t\t\t\t\t\t\t\r')
            else:
                elapsed_time = int(time.time() - start_time)
                print ('\rDuracion pros palabra:{}\t procesados:{},validados:{}'.format(elapsed_time,count,valid_count),end='\t\t\t\t\t\t\t\t\t\r')
            count+=1
        else:
            print('\n \n listo')
            break  

In [13]:
conditional_hsic_to_file(origen, destino, 3397, 3999)

Duracion pros palabra:30	 procesados:3999,validados:3									
 
 listo
